# Assignment
Lab04-2 110065508 李丞恩

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
if not os.path.exists('./output_HW'):
    os.makedirs('./output_HW')

In [ ]:
X_train = pd.read_csv('./data/X_train.csv')
y_train = pd.read_csv('./data/y_train.csv')

X_test = pd.read_csv('./data/X_test.csv')

In [ ]:
print(X_train.shape)
print(X_train.columns)
print(y_train.columns)

In [ ]:
X_train.head()